In [6]:
from os import listdir
import string

from gensim.models import Word2Vec

In [7]:
def load_doc(filename):
    with open(filename) as file:
        doc = file.read()
        return doc

def process_doc_for_w2v(doc, vocab):
    sentences = list()
    for line in doc.splitlines():
        table  = str.maketrans('', '', string.punctuation)
        line = line.translate(table)
        tokens = line.split()
        tokens = [word for word in tokens if word in vocab]
        sentences.append(tokens)
    return sentences

def load_dataset_w2v(directory, vocab, is_train = True):
    features = list()
    for file in listdir(directory):
        if is_train and file.startswith('cv9'):
            continue
        if is_train == False and not file.startswith('cv9'):
            continue
        with open('/'.join([directory, file])) as f:
            doc = f.read()
            sentences = process_doc_for_w2v(doc, vocab) 
            features += sentences
    return features 

In [8]:
#Train word 2 vec on dataset and produce word_2_vec.txt
vocab = load_doc('vocab.txt')
vocab = set(vocab.split())
print('vocab size = ', len(vocab))

pos_train = load_dataset_w2v('./datasets/movie_review/txt_sentoken/pos', vocab)
neg_train = load_dataset_w2v('./datasets/movie_review/txt_sentoken/neg', vocab)

sentences = pos_train + neg_train
print("training sentences: ", len(sentences))

w2v = Word2Vec(sentences, vector_size=100, window=5, workers=8, min_count=1)
words = list(w2v.wv.key_to_index)
print('Total number of words ',len(words))
filename = 'word_2_vec.txt'
w2v.wv.save_word2vec_format(filename, binary=False)

vocab size =  25435
training sentences:  58109
Total number of words  25435
